## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Открываем файл с данными и изучаем общую информацию.

#### Подключим библиотеки и прочтём файл

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

borrowers = pd.read_csv('/datasets/data.csv')

#### Изучим общую информацию

##### Расшифровка столбцов

*  children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

In [2]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


* Столбец с трудовым стажем имеет вещественный тип данных(округлим до целочисленного)
* У столбцов days_employed и purpose есть пропуски, и их количество равно (проверить совпадают ли строки с пропусками)


In [3]:
borrowers.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
borrowers.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


* Количество отработанных дней имеют отрицательные значения, но в общем на первый взгляд они выглядят адекватными (проверить на возможность взять числа по модулю, и для удобства чтения перевести в года)
* Столбец образование выглядит не однородно (привести к стандартному виду)
* Округление столбца с ежемесячным доходом до целых чисел, повысит читабельность


In [5]:
borrowers.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


* Смущает отрицательное количество детей (возможно лишний знак)
* 20 детей сильно выбивается из среднего значения (проверить верность данных)
* Среднее количество отработанных дней больше 170 лет, а максимальное более 1100 лет (попробовать найти соотношение ошибок с другими данными)
* Минимальный возраст 0 лет (колонка не важна для поставленных задач)

### Вывод 

Предполагаю что в колонке с количеством детей отрицательное значение можно исправить на положительное, и возможно 20 детей это тоже опечатка, которую можно исправить на 2. В колонке с трудовым стажем достаточно много непонятного: во-первых разобраться с отрицательными значениями, но вопрос с рекордсменами трудящимися более 100 лет остается, попробую сопоставить с возрастом, а возможно это всего несколько ошибок дают смещение(разберусь с отрицательными, и буду смотреть по медиане, если она адекватная найду рекордсменов и подправлю их).



### Шаг 2. Предобработка данных

### Обработка пропусков

#### Проверю совпадают ли строки с пропусками

In [6]:
borrowers.loc[borrowers['days_employed'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Строки совпадают. Я предположил что по этим людям нет информации о работе, но нет колонка с типом занятости заполнена. Пропусков 10% от общих данных значит я не могу ими пренебрегать, заменив на нули. Мы точно знаем что человек работает и имеет доход исходя из колонки с типом занятости. Буду подставлять среднее значение в доход, и медианное в стаж (в доходе нет такого гигантского разброса который может повлиять существенно на статистику, а в стаже есть "ударники труда").

Возьмем значение трудового стажа по модулю

In [7]:
borrowers['days_employed'] = borrowers['days_employed'].abs()


Посмотрел на тех у кого возраст равен нулю.(первым делом исправлю возраст пенсионеров, тут самые однозначные результаты)


In [8]:
pensioner_age = borrowers.loc[borrowers['income_type'] == 'пенсионер']['dob_years'].mean()
borrowers.loc[(borrowers['dob_years'] == 0) & (borrowers['income_type'] == 'пенсионер'), 'dob_years'] = pensioner_age


Дальше мне кажется самым показательным будет количество детей. Проанализировал соотношения количества детей и возраста, пришел к выводу, что разброс небольшой и прямой зависимости нет. Поэтому буду подставлять среднее значение исключив пенсионеров.


In [9]:
mean_age = borrowers.loc[borrowers['income_type'] != 'пенсионер']['dob_years'].mean()
borrowers.loc[borrowers['dob_years'] == 0,'dob_years'] = mean_age


Теперь буду исправлять пропуски в трудовом стаже. Сначала посмотрю сколько людей работали дольше чем живут.


In [10]:
borrowers.loc[borrowers['days_employed'] > (borrowers['dob_years'] * 365)]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64


Проблема в основном с пенсионерами, и всего двое безработных которые работали дольше чем живут. (Так же я попробовал добавить начало трудового стажа не с рождения, а с 18 лет, цифры не меняются. Я оставил отсчет с рождения.



Думал подменить стаж пенсионеров с аномальными значениями, на стаж пенсионеров с нормальными, но таких нет.


In [11]:
mean_56 = borrowers.loc[(borrowers['days_employed'] < (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] <= 56)]['days_employed'].mean()
mean_60 = borrowers.loc[(borrowers['days_employed'] < (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] > 56) & (borrowers['dob_years'] <= 60)]['days_employed'].mean()
mean_64 = borrowers.loc[(borrowers['days_employed'] < (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] > 60) & (borrowers['dob_years'] <= 64)]['days_employed'].mean()
mean_old = borrowers.loc[(borrowers['days_employed'] < (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] > 64)]['days_employed'].mean()                   


Разбил на 4 возрастные группы с равным количеством человек, с завышенными данными. Взял средние значения из аналогичных групп с нормальными данными.


In [12]:
borrowers.loc[(borrowers['days_employed'] > (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] <= 56), 'days_employed'] = mean_56                                                                            
borrowers.loc[(borrowers['days_employed'] > (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] >= 56) & (borrowers['dob_years'] < 60),'days_employed'] = mean_60
borrowers.loc[(borrowers['days_employed'] > (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] >= 60) & (borrowers['dob_years'] < 64),'days_employed'] = mean_64
borrowers.loc[(borrowers['days_employed'] > (borrowers['dob_years'] * 365)) & (borrowers['dob_years'] >=64),'days_employed'] = mean_old


То же самое сделаю с пропусками в стаже, разобью на 4 группы


In [13]:
mean_34 = borrowers.loc[(borrowers['days_employed'].notnull()) & (borrowers['dob_years'] <= 34),'days_employed'].mean()
mean_43 = borrowers.loc[(borrowers['days_employed'].notnull()) & (borrowers['dob_years'] > 34) & (borrowers['dob_years'] <= 43),'days_employed'].mean()
mean_54 = borrowers.loc[(borrowers['days_employed'].notnull()) & (borrowers['dob_years'] > 43) & (borrowers['dob_years'] <= 54),'days_employed'].mean()
mean_older = borrowers.loc[(borrowers['days_employed'].notnull()) & (borrowers['dob_years'] > 54),'days_employed'].mean()

borrowers.loc[(borrowers['days_employed'].isnull()) & (borrowers['dob_years'] <= 34),'days_employed'] = mean_34
borrowers.loc[(borrowers['days_employed'].isnull()) & (borrowers['dob_years'] > 34) & (borrowers['dob_years'] <= 43),'days_employed'] = mean_43
borrowers.loc[(borrowers['days_employed'].isnull()) & (borrowers['dob_years'] > 43) & (borrowers['dob_years'] <= 54),'days_employed'] = mean_54 
borrowers.loc[(borrowers['days_employed'].isnull()) & (borrowers['dob_years'] > 54),'days_employed'] = mean_older
borrowers.loc[borrowers['days_employed'].isnull()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose



Избавился от чрезмерно больших чисел в столбце со стажем, и убрал пропуски



Пропуски в доходе буду заполнять средними значениями по должности


In [14]:
employee_income = borrowers.loc[(borrowers['total_income'].notnull()) & (borrowers['income_type'] == 'сотрудник'), 'total_income'].mean()
partner_income = borrowers.loc[(borrowers['total_income'].notnull()) & (borrowers['income_type'] == 'компаньон'), 'total_income'].mean()
pensioner_income = borrowers.loc[(borrowers['total_income'].notnull()) & (borrowers['income_type'] == 'пенсионер'), 'total_income'].mean()
civil_servant_income = borrowers.loc[(borrowers['total_income'].notnull()) & (borrowers['income_type'] == 'госслужащий'), 'total_income'].mean()
entrepreneur_income = borrowers.loc[(borrowers['total_income'].notnull()) & (borrowers['income_type'] == 'предприниматель'), 'total_income'].mean()

borrowers.loc[(borrowers['total_income'].isnull()) & (borrowers['income_type'] == 'сотрудник'), 'total_income'] = employee_income
borrowers.loc[(borrowers['total_income'].isnull()) & (borrowers['income_type'] == 'компаньон'), 'total_income'] = partner_income
borrowers.loc[(borrowers['total_income'].isnull()) & (borrowers['income_type'] == 'пенсионер'), 'total_income'] = pensioner_income
borrowers.loc[(borrowers['total_income'].isnull()) & (borrowers['income_type'] == 'госслужащий'), 'total_income'] = civil_servant_income
borrowers.loc[(borrowers['total_income'].isnull()) & (borrowers['income_type'] == 'предприниматель'), 'total_income'] = entrepreneur_income
borrowers.loc[borrowers['total_income'].isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Посмотрим на трудоголиков

In [15]:
borrowers.sort_values('days_employed', ascending=False).head(20)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,18388.949901,61.0,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,17615.563266,61.0,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,16593.472817,60.0,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,16264.699501,59.0,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,16119.687737,64.0,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости
3974,0,15835.725775,64.0,среднее,1,гражданский брак,1,F,компаньон,0,96858.531436,сыграть свадьбу
1539,0,15785.678893,59.0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,119563.851852,операции с коммерческой недвижимостью
4321,0,15773.061335,61.0,среднее,1,гражданский брак,1,F,сотрудник,0,205868.585780,свадьба
7731,0,15618.063786,64.0,среднее,1,женат / замужем,0,F,компаньон,0,296525.358574,высшее образование
15675,0,15410.040779,65.0,высшее,0,женат / замужем,0,F,сотрудник,0,188800.068859,покупка жилой недвижимости


Первая гипотеза: что есть всего несколько уникальных строк с некорректными данными в стаже, не подтвердилась. Вторая гипотеза: что смещен порядок числа, тоже не подтвердилась
Так как этот столбец не влияет на вывод мы его пока не будем трогать

Проверю количество детей. Сгруппирую и исправлю аномалии.

In [16]:
borrowers['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

-1 ребенок явная опечатка исправляем на 1, 20 тоже опечатка иначе картина была бы более плавная, и мы наблюдали плавное изменение количества детей, а так как заемщиков с количеством детей от 6 до 19 не наблюдаем, мы можем быть уверенны, что и здесь ошибка(исправляем).

In [17]:
borrowers.loc[borrowers['children'] == -1, 'children'] = 1
borrowers.loc[borrowers['children'] == 20, 'children'] = 2
borrowers['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

1. Заменили пропуски в стаже на медианное значение из-за большого разброса в данных.
2. Заменили пропуски в доходе на среднее значение.
3. Заменили отрицательные значения в трудовом стаже
4. Исправил количество детей

### Замена типа данных

Заменим тип данных в столбцах: 'days_employed' и 'total_income' на целочисленный для удобства чтения

In [18]:
borrowers['days_employed'] = borrowers['days_employed'].astype('int64')
borrowers['total_income'] = borrowers['total_income'].astype('int64')
borrowers.dtypes

children              int64
days_employed         int64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

### Вывод

Заменили типы данных в столбцах содержащих трудовой стаж и доход.

### Обработка дубликатов

Приведу все текстовые столбцы к единому виду

In [19]:
borrowers['education'] = borrowers['education'].str.lower()
borrowers['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
borrowers['family_status'] = borrowers['family_status'].str.lower()
borrowers['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [21]:
borrowers['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

странные значения (посмотрим на них)

In [22]:
borrowers.loc[borrowers['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


Определить гендер по другим данным не представляется возможным, но для конечного вывода он нам не нужен (оставляем)

In [23]:
borrowers['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Посчитаем количество дубликатов.

In [24]:
borrowers.duplicated().sum()

71

In [25]:
borrowers = borrowers.drop_duplicates()

In [26]:
borrowers.duplicated().sum()

0

### Вывод

Привели все текстовые поля к единому стандарту, удалили дубликаты, и проверили.
Предполагаю что дубликаты возникли из-за за двоения данных

### Лемматизация

Добавим колонку и лемматизируем её, для удобства последующего анализа.

In [27]:
borrowers['purpose_lem'] = borrowers['purpose'].apply(lambda x: m.lemmatize(x))
borrowers['purpose_lem'].value_counts().to_frame()

,purpose_lem
"[автомобиль, \n]",972
"[свадьба, \n]",791
"[на, , проведение, , свадьба, \n]",768
"[сыграть, , свадьба, \n]",765
"[операция, , с, , недвижимость, \n]",675
"[покупка, , коммерческий, , недвижимость, \n]",661
"[операция, , с, , жилье, \n]",652
"[покупка, , жилье, , для, , сдача, \n]",651
"[операция, , с, , коммерческий, , недвижимость, \n]",650
"[покупка, , жилье, \n]",646


Можем выделить несколько категорий: автомобиль, недвижимость(жилье), свадьба, образование.

### Вывод

Добавил колонку с лемматезированными данными по цели кредита. При визуальном осмотре выделил 4 основные категории.

### Категоризация данных

Сгруппируем по количеству детей, и выведем просрочки по кредитам, и количество клиентов

In [28]:
children_and_debt = borrowers.groupby('children')['debt'].agg(['sum', 'count'])
children_and_debt['proportion'] = children_and_debt['sum'] / children_and_debt['count']
children_and_debt

,sum,count,proportion
children,,,
0,1063,14091,0.075438
1,445,4855,0.091658
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


Посчитали долю клиентов имевших задолженность в соотношение с количеством детей

Сгруппируем по семейному положению, и выведем количество клиентов в различном семейном положение, и количество просрочек

In [29]:
family_and_debt = borrowers.groupby('family_status')['debt'].agg(['sum', 'count'])
family_and_debt['proportion'] = family_and_debt['sum'] / family_and_debt['count']
family_and_debt

,sum,count,proportion
family_status,,,
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
не женат / не замужем,274,2810,0.097509


Вывели зависимость надежности клиентов в зависимости от семейного положения

Сгруппируем по уровню дохода

In [30]:
def income_sorted(total_income):
    
    if total_income <= 107623:
        return 'до 107623 '
    if total_income <= 156043:
        return 'до 156043'
    if total_income <= 195813:
        return 'до 195813'
    if 195813 < total_income:
        return 'от 195813'
    else:
        print('total_income')

income_group = borrowers.loc[:, ('debt','total_income')]
income_group['income_id'] = income_group['total_income'].apply(income_sorted)
income_and_debt = income_group.groupby('income_id')['debt'].agg(['sum', 'count'])
income_and_debt['proportion'] = income_and_debt['sum'] / income_and_debt['count']
income_and_debt

,sum,count,proportion
income_id,,,
до 107623,427,5364,0.079605
до 156043,503,5749,0.087493
до 195813,398,4475,0.088939
от 195813,413,5866,0.070406


Выделили 4 равные группы по количеству людей. Присвоили каждой группе индекс в зависимости от дохода.

Группируем по целям

In [31]:

def purpose_sorted(purpose_lem):
    if 'автомобиль' in purpose_lem:
        return 'автомобиль'
    if 'жилье' in purpose_lem or 'недвижимость' in purpose_lem:
        return 'недвижимость'
    if 'свадьба' in purpose_lem:
        return 'свадьба'
    if 'образование':
        return 'образование'
    else:
        print(purpose_lem)

purpose_group = borrowers.loc[:, ('debt','purpose_lem')]
purpose_group['purpose_id'] = purpose_group['purpose_lem'].apply(purpose_sorted)
purpose_and_debt = purpose_group.groupby('purpose_id')['debt'].agg(['sum', 'count'])
purpose_and_debt['proportion'] = purpose_and_debt['sum'] / purpose_and_debt['count']
purpose_and_debt

,sum,count,proportion
purpose_id,,,
автомобиль,403,4306,0.093590
недвижимость,782,10811,0.072334
образование,370,4013,0.092200
свадьба,186,2324,0.080034


### Вывод

### Шаг 3. Отвечаем на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
children_pivot = borrowers.pivot_table('debt', index='children', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())],)
children_pivot = children_pivot.rename(columns={"sum":"количество должников", "count":"количество заемщиков", "<lambda>": "доля должников", "debt": "" })
children_pivot.index.name = 'количество детей'
children_pivot          

,количество должников,количество заемщиков,доля должников
,,,
количество детей,,,
0,1063,14091,7.54%
1,445,4855,9.17%
2,202,2128,9.49%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


### Вывод

Между наличием детей и возвратом кредита зависимость есть. Так же наблюдается корреляция между количеством детей и вероятностью возврата кредита. Чаще всего кредиты возвращают люди не имеющие детей, а реже всего: люди с двумя детьми. По клиентам с количеством детей больше трех недостаточно данных для выводов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
family_pivot = borrowers.pivot_table('debt', index='family_status', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())],)
family_pivot = family_pivot.rename(columns={"sum":"количество должников", "count":"количество заемщиков", "<lambda>": "доля должников", "debt": "" })
family_pivot.index.name = 'семейное положение'
family_pivot  

,количество должников,количество заемщиков,доля должников
,,,
семейное положение,,,
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%
гражданский брак,388,4151,9.35%
женат / замужем,931,12339,7.55%
не женат / не замужем,274,2810,9.75%


### Вывод

Семейное положение влияет на возврат кредита. У не женатых ,и живущих в гражданском браке шанс возврата кредита ниже. Самые надежные заемщики вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
income_pivot = income_group.pivot_table('debt', index='income_id', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())],)
income_pivot = income_pivot.rename(columns={"sum":"количество должников", "count":"количество заемщиков", "<lambda>": "доля должников", "debt": "" })
income_pivot.index.name = 'доход'
income_pivot  

,количество должников,количество заемщиков,доля должников
,,,
доход,,,
до 107623,427,5364,7.96%
до 156043,503,5749,8.75%
до 195813,398,4475,8.89%
от 195813,413,5866,7.04%


### Вывод

На шанс возврата кредита влияет уровень дохода. Самый высокий шанс вернуть кредит у клиентов с доходом свыше 195 813.

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
purpose_pivot = purpose_group.pivot_table('debt', index='purpose_id', aggfunc=['sum', 'count', lambda x: '{:.2%}'.format(x[:].mean())],)
purpose_pivot = purpose_pivot.rename(columns={"sum":"количество должников", "count":"количество заемщиков", "<lambda>": "доля должников", "debt": "" })
purpose_pivot.index.name = 'цель кредита'
purpose_pivot 

,количество должников,количество заемщиков,доля должников
,,,
цель кредита,,,
автомобиль,403,4306,9.36%
недвижимость,782,10811,7.23%
образование,370,4013,9.22%
свадьба,186,2324,8.00%


### Вывод

Цели кредита также влияют на шанс возврата кредита. Так больше вероятность вернуть кредит у заемщиков берущих кредит на недвижимость, а те кто брал деньги на автомобиль или образование, мене склонны возвращать кредит

### Шаг 4. Общий вывод

На шанс вернуть кредит влияет семейное положение и количество детей:
  - Так самыми надежными заемщиками являются те кто вообще не имеет детей, они не возвращают кредит всего в 7.54%, в отличие от клиентов имеющих двое детей, которые не так добросовестно относятся к задолженности и не возвращают в 9.49% случаев.
  - Что касается семейного положения то клиенты не состоящие в браке, не платят по кредитам в 9.75% случаев. А вот вдовцы самая надежная категория, кредиторов 6.57% уклоняются от выплат.
Так же есть зависимость между возвратом кредита, и целями, и доходом заемщика:
  - По целям самые добросовестные заемщики те кто берет деньги на недвижимость, они не выплачивают в 7.23% случаев. Заёмщики берущие деньги на автомобиль не платят в 9.36% случаев.
  - Клиенты с доходом в диапазоне от 156044 до 195813 не платят по кредиту 8.89% случаях. А те у кого доход свыше 195813 не выплачивают в 7.04%.


  - Рекомендую проанализировать зависимость с возрастом возможно он в большей степени влияет на шанс возврата денег и сказывается на остальной статистике, так я могу предположить что вдовцы и люди с большим количеством детей, чаще будут в более высокой возрастной категории, чем холостые. Так же я не смог придумать как с высокой достоверностью восстановить данные из колонки трудового стажа. Хотелось бы более подробно узнать как собирались эти данные, возможно ошибка конвертации.

hh
  - h